In [ ]:
Import necessary packages, read data, remove unnecessary columns

In [1]:
import pandas as pd
import csv
from pathlib import Path
import os

#cleaning data
schedule = pd.read_csv("reg_season.csv")
schedule.head()
schedule = schedule.iloc[:,1:-2] #dropping the first and last two columns
schedule.head()

advanced_stats = pd.read_csv("advanced_stats.csv")
advanced_stats.head()
advanced_stats = advanced_stats.dropna(axis=1, how ='all') #remove NaN columns
advanced_stats = advanced_stats.iloc[:, 1:-1]#remove first and last column
advanced_stats.head()



,Team,Age,W,L,PW,PL,MOV,SOS,SRS,ORtg,...,3PAr,TS%,eFG%,TOV%,ORB%,FT/FGA,eFG%.1,TOV%.1,DRB%,FT/FGA.1
0,Las Vegas Aces,29.0,34.0,6.0,35,5,12.55,-1.09,11.46,114.8,...,0.360,0.597,0.553,12.5,18.9,0.236,0.480,14.4,78.2,0.171
1,New York Liberty,28.8,32.0,8.0,32,8,8.68,-0.53,8.14,111.8,...,0.424,0.577,0.539,14.9,25.8,0.198,0.476,13.1,78.5,0.184
2,Connecticut Sun,29.1,27.0,13.0,26,14,3.70,-0.27,3.43,105.4,...,0.294,0.540,0.498,14.0,23.9,0.224,0.486,16.7,75.8,0.236
3,Dallas Wings,26.0,22.0,18.0,25,15,2.95,-0.31,2.64,109.5,...,0.292,0.535,0.489,13.7,32.8,0.224,0.504,15.0,77.2,0.237
4,Washington Mystics,28.8,19.0,21.0,19,21,-0.35,0.03,-0.32,101.5,...,0.342,0.533,0.486,13.9,18.7,0.222,0.496,16.3,78.0,0.238


In [2]:
#merging the two dataframes by team name 
df = pd.merge(schedule, advanced_stats, left_on="Visitor/Neutral", right_on="Team")
df = pd.merge(df, advanced_stats, left_on="Home/Neutral", right_on="Team")
df.head()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Team_x,Age_x,W_x,L_x,PW_x,PL_x,...,3PAr_y,TS%_y,eFG%_y,TOV%_y,ORB%_y,FT/FGA_y,eFG%.1_y,TOV%.1_y,DRB%_y,FT/FGA.1_y
0,Connecticut Sun,70,Indiana Fever,61,Connecticut Sun,29.1,27.0,13.0,26,14,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
1,Connecticut Sun,88,Indiana Fever,72,Connecticut Sun,29.1,27.0,13.0,26,14,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
2,Phoenix Mercury,85,Indiana Fever,82,Phoenix Mercury,27.8,9.0,31.0,8,32,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
3,Phoenix Mercury,71,Indiana Fever,72,Phoenix Mercury,27.8,9.0,31.0,8,32,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
4,Chicago Sky,89,Indiana Fever,87,Chicago Sky,27.2,18.0,22.0,17,23,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248


In [3]:
#get rid of duplicate columns
df = df.drop(["Team_x","Team_y"], axis=1)
df.head()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Age_x,W_x,L_x,PW_x,PL_x,MOV_x,...,3PAr_y,TS%_y,eFG%_y,TOV%_y,ORB%_y,FT/FGA_y,eFG%.1_y,TOV%.1_y,DRB%_y,FT/FGA.1_y
0,Connecticut Sun,70,Indiana Fever,61,29.1,27.0,13.0,26,14,3.70,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
1,Connecticut Sun,88,Indiana Fever,72,29.1,27.0,13.0,26,14,3.70,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
2,Phoenix Mercury,85,Indiana Fever,82,27.8,9.0,31.0,8,32,-8.30,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
3,Phoenix Mercury,71,Indiana Fever,72,27.8,9.0,31.0,8,32,-8.30,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
4,Chicago Sky,89,Indiana Fever,87,27.2,18.0,22.0,17,23,-1.63,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248


In [4]:
#identify winning team based on who scored more points
for index, row in df.iterrows(): 
    if df.loc[index, "PTS"] > df.loc[index, "PTS.1"]:
        df.loc[index, "Home_winner"] = 0
    if df.loc[index, "PTS"] < df.loc[index, "PTS.1"]:
        df.loc[index, "Home_winner"] = 1

df.head()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Age_x,W_x,L_x,PW_x,PL_x,MOV_x,...,TS%_y,eFG%_y,TOV%_y,ORB%_y,FT/FGA_y,eFG%.1_y,TOV%.1_y,DRB%_y,FT/FGA.1_y,Home_winner
0,Connecticut Sun,70,Indiana Fever,61,29.1,27.0,13.0,26,14,3.70,...,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248,0.0
1,Connecticut Sun,88,Indiana Fever,72,29.1,27.0,13.0,26,14,3.70,...,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248,0.0
2,Phoenix Mercury,85,Indiana Fever,82,27.8,9.0,31.0,8,32,-8.30,...,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248,0.0
3,Phoenix Mercury,71,Indiana Fever,72,27.8,9.0,31.0,8,32,-8.30,...,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248,1.0
4,Chicago Sky,89,Indiana Fever,87,27.2,18.0,22.0,17,23,-1.63,...,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248,0.0


In [5]:
#selecting all columns we haven't used
remove_cols = ["Home/Neutral", "Visitor/Neutral", "PTS", "PTS.1", "Home_winner"]
selected_cols = [x for x in df.columns if x not in remove_cols]
df[selected_cols].head()

,Age_x,W_x,L_x,PW_x,PL_x,MOV_x,SOS_x,SRS_x,ORtg_x,DRtg_x,...,3PAr_y,TS%_y,eFG%_y,TOV%_y,ORB%_y,FT/FGA_y,eFG%.1_y,TOV%.1_y,DRB%_y,FT/FGA.1_y
0,29.1,27.0,13.0,26,14,3.70,-0.27,3.43,105.4,100.7,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
1,29.1,27.0,13.0,26,14,3.70,-0.27,3.43,105.4,100.7,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
2,27.8,9.0,31.0,8,32,-8.30,0.55,-7.75,99.1,109.9,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
3,27.8,9.0,31.0,8,32,-8.30,0.55,-7.75,99.1,109.9,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248
4,27.2,18.0,22.0,17,23,-1.63,0.11,-1.52,103.3,105.3,...,0.29,0.532,0.491,15.6,26.4,0.204,0.509,13.9,76.8,0.248


In [6]:
#fitting the selected columns to allow for feature selection
from sklearn.preprocessing import MinMaxScaler
scalar = MinMaxScaler()
df[selected_cols] = scalar.fit_transform(df[selected_cols])
df.head()

,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Age_x,W_x,L_x,PW_x,PL_x,MOV_x,...,TS%_y,eFG%_y,TOV%_y,ORB%_y,FT/FGA_y,eFG%.1_y,TOV%.1_y,DRB%_y,FT/FGA.1_y,Home_winner
0,Connecticut Sun,70,Indiana Fever,61,1.00000,0.72,0.28,0.666667,0.333333,0.575540,...,0.177215,0.225,0.659574,0.546099,0.392405,0.891892,0.216216,0.690909,0.9625,0.0
1,Connecticut Sun,88,Indiana Fever,72,1.00000,0.72,0.28,0.666667,0.333333,0.575540,...,0.177215,0.225,0.659574,0.546099,0.392405,0.891892,0.216216,0.690909,0.9625,0.0
2,Phoenix Mercury,85,Indiana Fever,82,0.59375,0.00,1.00,0.000000,1.000000,0.000000,...,0.177215,0.225,0.659574,0.546099,0.392405,0.891892,0.216216,0.690909,0.9625,0.0
3,Phoenix Mercury,71,Indiana Fever,72,0.59375,0.00,1.00,0.000000,1.000000,0.000000,...,0.177215,0.225,0.659574,0.546099,0.392405,0.891892,0.216216,0.690909,0.9625,1.0
4,Chicago Sky,89,Indiana Fever,87,0.40625,0.36,0.64,0.333333,0.666667,0.319904,...,0.177215,0.225,0.659574,0.546099,0.392405,0.891892,0.216216,0.690909,0.9625,0.0


In [7]:
#selecting the best 10 features 
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1.0)
sfs = SequentialFeatureSelector(rr, n_features_to_select=10, direction="backward")

In [8]:
sfs.fit(df[selected_cols], df["Home_winner"])

SequentialFeatureSelector(direction='backward', estimator=RidgeClassifier(),
                          n_features_to_select=10)

In [9]:
predictors = list(df[selected_cols].columns[sfs.get_support()])
df[predictors].head()

,ORB%_x,eFG%.1_x,TOV%.1_x,L_y,TS%_y,eFG%_y,ORB%_y,FT/FGA_y,eFG%.1_y,TOV%.1_y
0,0.368794,0.270270,0.972973,0.84,0.177215,0.225,0.546099,0.392405,0.891892,0.216216
1,0.368794,0.270270,0.972973,0.84,0.177215,0.225,0.546099,0.392405,0.891892,0.216216
2,0.156028,0.810811,0.081081,0.84,0.177215,0.225,0.546099,0.392405,0.891892,0.216216
3,0.156028,0.810811,0.081081,0.84,0.177215,0.225,0.546099,0.392405,0.891892,0.216216
4,0.404255,0.621622,0.405405,0.84,0.177215,0.225,0.546099,0.392405,0.891892,0.216216


In [10]:
#using the predictors to train a model using logistic regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def monte_carlo(n):
    accuracy = []
    for i in range(n):
        #split the data into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(df[predictors], df['Home_winner'], test_size=0.2)

        #train a logistic regression model on the training data
        model = LogisticRegression()
        
        # Fit the model to our training data
        model.fit(X_train, y_train)

        #predict the winners for the test data
        y_pred = model.predict(X_test)

        #evaluate the accuracy of the model on the test data
        accuracy.append(accuracy_score(y_test, y_pred))

    #get the average accuracy
    score = sum(accuracy) / len(accuracy)
    return score

In [11]:
score = monte_carlo(1000)
print(f"Accuracy: {score}") # about 70% accuracy, pretty good!

Accuracy: 0.705416666666666


In [ ]:
#remove the rows where the Aces and Liberty play against each other
non_aces_liberty_game = df
non_aces_liberty_game = non_aces_liberty_game.drop(non_aces_liberty_game[(non_aces_liberty_game['Home/Neutral'] == 'Las Vegas Aces') & (non_aces_liberty_game['Visitor/Neutral'] == 'New York Liberty')].index)
non_aces_liberty_game = non_aces_liberty_game.drop(non_aces_liberty_game[(non_aces_liberty_game['Home/Neutral'] == 'New York Liberty') & (non_aces_liberty_game['Visitor/Neutral'] == 'Las Vegas Aces')].index)
non_aces_liberty_game[(non_aces_liberty_game['Home/Neutral'] == 'Las Vegas Aces') & (non_aces_liberty_game['Visitor/Neutral'] == 'New York Liberty')]

,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Age_x,W_x,L_x,PW_x,PL_x,MOV_x,...,TS%_y,eFG%_y,TOV%_y,ORB%_y,FT/FGA_y,eFG%.1_y,TOV%.1_y,DRB%_y,FT/FGA.1_y,Home_winner


In [13]:
#get a game where the Aces are home and Liberty is away
final_matchup = df[(df['Home/Neutral'] == 'Las Vegas Aces') & (df['Visitor/Neutral'] == 'New York Liberty')][:1]
#show the predictors we will use
final_matchup[predictors]

,ORB%_x,eFG%.1_x,TOV%.1_x,L_y,TS%_y,eFG%_y,ORB%_y,FT/FGA_y,eFG%.1_y,TOV%.1_y
144,0.503546,0.0,0.0,0.0,1.0,1.0,0.014184,0.797468,0.108108,0.351351


In [15]:
#predict the winner of the final matchup
model = LogisticRegression()
model.fit(non_aces_liberty_game[predictors], non_aces_liberty_game['Home_winner'])

#predict the outcome of the final_matchup
y_pred = model.predict(final_matchup[predictors])
print(f"Prediction: {y_pred[0]}")

Prediction: 1.0
